# Symbolic algebra

## Introduction

First, we need to import a library for symbolic computation. We'll be using
`sympy` for this purpose. We'll also import `numpy` for a few comparisons.

I recommend avoiding `from sympy import *` (even though this is used in the
[`sympy` tutorial](http://docs.sympy.org/latest/tutorial/)), since we will
want to use some functions, like `sqrt`, from both a symbolic and a numeric
library.

Sometimes we have an exact expression that is corrupted by
the numerical imprecision of a library like `numpy`. For example,
consider $\sqrt{3}$. `numpy` gives us a floating point approximation that doesn't
have all the properties we might like:

`sympy` gives us an object that has the properties we want:

`sympy` is also capable of displaying expressions in a more human-friendly
format.

The real power of `sympy` is the ability to manipulate variables. Some computer algebra
systems (CAS) allow the user to immediately input expressions with variables.

Since `sympy` is used within a full Python programming environment, we need to do a little
setup first and define some *Python* variables to function as our symbolic variables.

## Expressions

We can make all kinds of expressions in `sympy` using both arithmetic operators from Python
(to which `sympy` gives special meaning) and functions that `sympy` provides for us:

Just like with `sqrt`, `sympy` can hold exact values of `sin`, whereas `numpy` gives a floating-point
approximation:

### Simplification

A lot of times we want to manipulate an expression without changing what it represents. `sympy`
has a variety of functions for doing this. `simplify` is the most generic:

Sometimes `sympy` doesn't automatically know what we mean by "simple", so there are functions to
put expressions in particular forms. For instance, we might want to expand $(x+y)(x-y)$ to
$x^2-y^2$:

We might also want to go the other way:

See more examples in the tutorial section
[Simplification](http://docs.sympy.org/latest/tutorial/simplification.html).

### Example: checking a phase

Sometimes we want to verify properties of our expressions, e.g. show that the
magnitude of a complex phase is 1. Let's set up some real variables $a$ and $b$
for the real and imaginary parts of a complex number $c=a+ib$:

`sympy` knows that $a$ and $b$ are real, so they remain unchanged when we take their complex conjugates:

Let's express the phase of $a+bi$ using `sympy`:

A complex phase $\omega$ has the property that $\omega\omega^*=1$. Let's see if
our expression satisfies that property:

`sympy` isn't realizing that $a^2+b^2$ is positive, and that therefore the conjugation doesn't do
anything. We can take `sympy` by the hand and manually point this out, first defining a
pair of expressions we know are equivalent:

Then we can use `subs` to perform a substitution within our expression:

Sometimes you can force `sympy` to make simplifications that may note be justified
in general. For instance, you might think $x^zy^z$ should be the same as $(xy)^z$. It
is when $x,y,z$ are real, but `sympy` doesn't currently know they're real:

We can tell `sympy` to make simplifications that aren't justified in general by
supplying `force=True` to the `powsimp` function:

### Displaying expressions nicely

Sometimes we want to display our expressions with a little more context that `sympy` spits out.
We can use the rich HTML rendering system of jupyter notebooks together with the `latex`
export functionality provided by `sympy` to make our notebooks legible.

`latex` converts a `sympy` expression to LaTeX code:

If we want it rendered prettily, we need to wrap it in some math environment and explicitly use the
`HTML` function to format it for our notebook:

This is a little more verbose, but it allows us greater flexibilitym for instance adding
context to printing out the value of $c$:

We can even use advanced environments like align, but it will be helpful
to use raw string literals so we don't have to worry about all the `\`s:

You can also use this LaTeX in your paper. If you want to save the result of a calculation for
future manipulation, however, it's best not to use LaTeX (since it can be ambiguous). `sympy`'s
`srepr` function is better suited for this:

You can use `sympify` (not to be confused with `simplify`!) to convert an `srepr` string back to an expression:

If you loaded an expression that has symbols you haven't defined yet,
you can get the new symbols from the expression in a list:

You can also use [pickle](https://docs.python.org/3/library/pickle.html). I prefer
`srepr`, since although it's increadibly verbose it is in principle interpretable by
a human or other program without needing Python.

## Anupam's calculus sections

## Finding eigenvalues of matrices

### Tensor products

Let's say I want to know the eigenvalues of $a_H+a_H^\dagger$, where
$a_H=\sqrt{s^2+1}\,\sigma_-^{(1)}-s\,\sigma_+^{(2)}$.

First let's define the standard single-qubit vectors and operators $\left|g\right\rangle$,
$\left|e\right\rangle$, $\sigma_-=\left|g\middle\rangle\middle\langle e\right|$,
$\sigma_+=\sigma_-^\dagger$:

We need a way to deal with tensor product structure for $\sigma_\pm^{(n)}$.
Fortunately the `physics.quantum` module provides support for this:

Then we make a symbol for $s$ and build $a_H=\sqrt{s^2+1}\,\sigma_-^{(1)}-s\,\sigma_+^{(2)}$
and $X_H=a_H+a_H^\dagger$:

We can display $X_H$ with some context using `HTML`:

Now we are ready to solve for eigenvalues and eigenvectors:

### State update

Suppose we have Kraus operators $K_\pm=\frac{1}{2}(I\pm\epsilon\sigma_z)$,
and we want to know what our updated state

\begin{align}
    \rho_\pm&=\frac{K_\pm\rho K_\pm^\dagger}{\operatorname{tr}[K_\pm^\dagger K_\pm\rho]}
\end{align}

will look like.

First let's define Pauli operators

\begin{align}
    \sigma_x&=\sigma_++\sigma_- \\
    \sigma_y&=-i\sigma_++i\sigma_- \\
    \sigma_z&=I-2\sigma_-\sigma_+
\end{align}

Then build $\rho$ and $K_\pm$ out of these operators.

Finally, calculate the state update:

If $\epsilon$ is small, we might only want to keep track of terms up to a certain
order in $\epsilon$. Let's expand the denominator to second order in $\epsilon$
using `series`:

The numerator is already only second order in $\epsilon$:

When we multiply the two terms together, the $\mathcal{O}(\epsilon^3)$ term swallows up
higher order products automatically:

We can also check properties of our updated state, such as its trace:

## Anupam's limit section